### Install The needed dependencies

The kagglehub library provides a simple way to interact with Kaggle resources such as datasets, models, notebook outputs in Python. This library also integrates natively with the Kaggle notebook environment

In [15]:
pip install kagglehub

StatementMeta(, 6229fd87-497f-4bae-9d2d-b2d92dfb090c, 22, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


The code below Authenticates Kaggle access using credentials, downloads the dataset, locates the CSV file, and loads it into a DataFrame for preliminary inspection.

In [16]:
import kagglehub
import pandas as pd
import os

# Set up credentials
df_creds = spark.read.option("multiline", "true").json("Files/kaggle.json")
creds_row = df_creds.collect()[0]

os.environ['KAGGLE_USERNAME'] = creds_row['username']
os.environ['KAGGLE_KEY'] = creds_row['key']

# Download without specifying a specific file
path = kagglehub.dataset_download("asaniczka/amazon-uk-products-dataset-2023")

# Find and load the CSV file
import glob
csv_files = glob.glob(os.path.join(path, "*.csv"))

if csv_files:
    df = pd.read_csv(csv_files[0])
    
    print(f"Loaded file: {os.path.basename(csv_files[0])}")
    print("Dataset shape:", df.shape)
    print("\nColumn names:")
    print(df.columns.tolist())
    
    # Show first 5 rows with all columns visible
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 100)
    
    print("\nFirst 5 records:")
    display(df.head())

StatementMeta(, 6229fd87-497f-4bae-9d2d-b2d92dfb090c, 23, Finished, Available, Finished)

 89%|████████▉ | 117M/131M [00:08<00:00, 17.7MB/s] 


Loaded file: amz_uk_processed_data.csv
Dataset shape: (2222742, 10)

Column names:
['asin', 'title', 'imgUrl', 'productURL', 'stars', 'reviews', 'price', 'isBestSeller', 'boughtInLastMonth', 'categoryName']

First 5 records:


SynapseWidget(Synapse.DataFrame, 10e181e6-0d07-4ea0-a587-735173445e92)

### Exploratory Data Analysis

In [17]:
df.sample

StatementMeta(, 6229fd87-497f-4bae-9d2d-b2d92dfb090c, 24, Finished, Available, Finished)

<bound method NDFrame.sample of                asin  \
0        B09B96TG33   
1        B01HTH3C8S   
2        B09B8YWXDF   
3        B09B8T5VGV   
4        B09WX6QD65   
...             ...   
2222737  B0CHSD7P2Q   
2222738  B0B7B8FNBQ   
2222739  B0BD5FL5FB   
2222740  B008OYLL4I   
2222741  B08P3W9B3T   

                                                                                                       title  \
0        Echo Dot (5th generation, 2022 release) | Big vibrant sound Wi-Fi and Bluetooth smart speaker wi...   
1        Anker Soundcore mini, Super-Portable Bluetooth Speaker with 15-Hour Playtime, 66-Foot Bluetooth ...   
2        Echo Dot (5th generation, 2022 release) | Big vibrant sound Wi-Fi and Bluetooth smart speaker wi...   
3        Echo Dot with clock (5th generation, 2022 release) | Bigger vibrant sound Wi-Fi and Bluetooth sm...   
4          Introducing Echo Pop | Full sound compact Wi-Fi and Bluetooth smart speaker with Alexa | Charcoal   
...                

In [18]:
df.shape

StatementMeta(, 6229fd87-497f-4bae-9d2d-b2d92dfb090c, 25, Finished, Available, Finished)

(2222742, 10)

Checking for duplicates

In [21]:
#check for duplicated
df.duplicated().sum()

StatementMeta(, 6229fd87-497f-4bae-9d2d-b2d92dfb090c, 28, Finished, Available, Finished)

0

In [23]:
df.drop_duplicates(inplace=True)

StatementMeta(, 6229fd87-497f-4bae-9d2d-b2d92dfb090c, 30, Finished, Available, Finished)

Checking data shape after removing duplicates

In [25]:
df.shape

StatementMeta(, 6229fd87-497f-4bae-9d2d-b2d92dfb090c, 32, Finished, Available, Finished)

(2222742, 10)

In [26]:
df.columns

StatementMeta(, 6229fd87-497f-4bae-9d2d-b2d92dfb090c, 33, Finished, Available, Finished)

Index(['asin', 'title', 'imgUrl', 'productURL', 'stars', 'reviews', 'price',
       'isBestSeller', 'boughtInLastMonth', 'categoryName'],
      dtype='object')

In [27]:
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

StatementMeta(, 6229fd87-497f-4bae-9d2d-b2d92dfb090c, 34, Finished, Available, Finished)

In [29]:
df.isnull().sum()

StatementMeta(, 6229fd87-497f-4bae-9d2d-b2d92dfb090c, 36, Finished, Available, Finished)

asin                 0
title                0
imgUrl               0
productURL           0
stars                0
reviews              0
price                0
isBestSeller         0
boughtInLastMonth    0
categoryName         0
dtype: int64

In [32]:
df['stars'].unique()

StatementMeta(, 6229fd87-497f-4bae-9d2d-b2d92dfb090c, 39, Finished, Available, Finished)

array([4.7, 4.6, 4.5, 4.8, 0. , 4.4, 4.3, 4.1, 4.9, 5. , 3.8, 3.6, 4. ,
       4.2, 3. , 3.9, 2.5, 3.4, 3.7, 3.2, 3.3, 3.5, 2.7, 2.1, 1. , 3.1,
       2. , 2.9, 2.8, 1.5, 2.2, 2.3, 2.6, 2.4, 1.4, 1.9, 1.8, 1.7, 1.6,
       1.2, 1.3, 1.1])

In [34]:
df['categoryName'].value_counts()

StatementMeta(, 6229fd87-497f-4bae-9d2d-b2d92dfb090c, 41, Finished, Available, Finished)

categoryName
Sports & Outdoors                 826076
Skin Care                          18755
Fragrances                         18563
Make-up                            18453
Manicure & Pedicure Products       17212
                                   ...  
Torches                               64
Smart Home Security & Lighting        57
Lighting                              50
Smart Speakers                        25
Cameras                               12
Name: count, Length: 296, dtype: int64

##### Converts the Pandas DataFrame to Spark and saves it as a managed Spark SQL table for scalable processing and querying.

In [43]:
# Convert pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(df)

# Now you can use Spark write operations
spark_df.write \
  .mode("overwrite") \
  .option("overwriteSchema", "true") \
  .saveAsTable("amazon_uk_products")

StatementMeta(, 6229fd87-497f-4bae-9d2d-b2d92dfb090c, 50, Finished, Available, Finished)